In [14]:
import tensorflow as tf
from tensorflow.keras import models, layers, regularizers
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder


import sys
sys.path.append("..")
from utils.preprocess import *

import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Bidirectional
from sklearn.model_selection import train_test_split


In [2]:
exported_path = '../exported_models/'

In [3]:
input_directory = "../assets/Respeck"

data_list = []
label_list = []
total_rows = 0

all_occu = {}

for folder_name in os.listdir(input_directory):
    folder_path = os.path.join(input_directory, folder_name)
    for file in os.listdir(folder_path):
        if file.endswith(".csv") and "unprocessed" not in file:
            
            filename = os.path.join(folder_path, file)
            label, _, _ = extract_activity_and_status(filename)
            if label not in all_occu:
                all_occu["label"] = 1
                all_occu["directory"] = [filename]
            else:
                all_occu["label"] += 1
                all_occu["directory"].append(filename)
            df = pd.read_csv(filename, usecols=[2,3,4])

            # Determine the number of rows to take from this file
            rows_to_take = min(700 - total_rows, len(df))
            
            # Update the df to only contain the necessary rows and update our counter
            df = df.head(rows_to_take)
            label_list.append(label)
            data_list.append(df)

# all_data = pd.concat(data_list, ignore_index=True)
print(data_list)

[      accel_x   accel_y   accel_z
0   -0.022461 -0.793274 -0.055969
1   -0.047852 -0.835754 -0.030579
2    0.055664 -0.816956 -0.071838
3    0.069336 -0.806213 -0.053528
4    0.084717 -0.867981  0.012146
..        ...       ...       ...
695 -0.020020 -0.749817 -0.120911
696 -0.028076 -0.736389 -0.053284
697 -0.000488 -0.906555  0.102722
698  0.042969 -0.902161  0.041687
699  0.143555 -0.840149  0.078308

[700 rows x 3 columns],       accel_x   accel_y   accel_z
0   -0.298828 -0.513733  0.028015
1   -0.401855 -0.663147  0.003601
2   -0.811035 -1.150208  0.144958
3   -0.756104 -1.152893 -0.061829
4    0.399658 -0.822815 -0.056458
..        ...       ...       ...
695  0.011475 -1.032532 -0.165344
696 -0.403564 -1.474426 -0.069885
697 -0.577393 -1.518860  0.009705
698 -0.357666 -1.091370  0.026550
699 -0.066895 -0.770569 -0.022766

[700 rows x 3 columns],       accel_x   accel_y   accel_z
0   -0.820312  0.136902  0.784607
1   -0.552490  0.143982  0.817810
2   -0.693359  0.140564  0.9506

In [4]:
print(len(data_list))

2024


In [5]:
for a in set(label_list):
    print(a)

standing
lying down back
lying down on stomach
running
descending stairs
sitting
lying down right
shuffle walking
ascending stairs
lying down on left
normal walking
miscellaneous movements


In [6]:
label_list[0:42]

['ascending stairs',
 'descending stairs',
 'lying down back',
 'lying down back',
 'lying down back',
 'lying down back',
 'lying down back',
 'lying down back',
 'lying down on left',
 'lying down on left',
 'lying down on left',
 'lying down on left',
 'lying down on left',
 'lying down on left',
 'lying down on stomach',
 'lying down on stomach',
 'lying down on stomach',
 'lying down on stomach',
 'lying down on stomach',
 'lying down on stomach',
 'lying down right',
 'lying down right',
 'lying down right',
 'lying down right',
 'lying down right',
 'lying down right',
 'miscellaneous movements',
 'normal walking',
 'running',
 'shuffle walking',
 'sitting',
 'sitting',
 'sitting',
 'sitting',
 'sitting',
 'sitting',
 'sitting',
 'standing',
 'standing',
 'standing',
 'standing',
 'standing']

In [7]:

# Normalizing sensor readings
scaler = StandardScaler()

sensor_columns = ['accel_x', 'accel_y', 'accel_z']

scaler.fit(data_list[0][sensor_columns])

for i in range(len(data_list)):
    data_list[i][sensor_columns] = scaler.transform(data_list[i][sensor_columns])


In [43]:
# import numpy as np
# from sklearn.model_selection import train_test_split

# # Convert data and labels to numpy arrays
# X = np.array([df.values for df in data_list])
# X = np.stack(X)
# y = LabelEncoder().fit_transform(label_list)

# # Split data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# # Convert X_train to float32
# X_train = X_train.astype('float32')

In [20]:
# Assuming your data_list and label_list are predefined
# Convert data and labels to numpy arrays
X = np.array([df.values for df in data_list])
X = np.stack(X)
y = LabelEncoder().fit_transform(label_list)

# Convert X to float32 and y to categorical (for softmax)
X = X.astype('float32')
y = to_categorical(y)

# Function to create the GRU model
# def create_model(input_shape, num_classes):
#     model = models.Sequential([
#         layers.GRU(128, input_shape=input_shape, return_sequences=True),  # GRU layer with 128 units
#         layers.GRU(64, return_sequences=False),  # Another GRU layer with 64 units
#         layers.Dense(64, activation='relu'),  # First fully connected layer
#         layers.Dense(32, activation='relu'),  # Second fully connected layer
#         layers.Dense(num_classes, activation='softmax')  # Output layer
#     ])
#     model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#     return model

def create_model(input_shape, num_classes):
    model = models.Sequential([
        # CNN layers for spatial feature extraction
        layers.Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape),
        layers.Conv1D(filters=64, kernel_size=3, activation='relu'),
        # Additional CNN layers
        layers.Conv1D(filters=128, kernel_size=3, activation='relu'),
        layers.Conv1D(filters=128, kernel_size=3, activation='relu'),
        layers.MaxPooling1D(pool_size=2),
        layers.BatchNormalization(),

        # Global pooling layer to reduce dimensionality while keeping the time dimension
        layers.GlobalMaxPooling1D(),
        # Repeat vector to add a 'time step' dimension
        layers.RepeatVector(3),

        #  # GRU layers
        # layers.GRU(128, return_sequences=True, 
        #            kernel_regularizer=regularizers.l2(0.001)),  # L2 Regularization
        # layers.GRU(64, kernel_regularizer=regularizers.l2(0.001)),  # L2 Regularization
        # layers.BatchNormalization(),
        # layers.Dropout(0.5),  # Dropout

        # LSTM layers
        # layers.LSTM(128, return_sequences=True, 
        #             kernel_regularizer=regularizers.l2(0.001)),  # L2 Regularization
        # layers.LSTM(64, kernel_regularizer=regularizers.l2(0.001)),  # L2 Regularization
        layers.LSTM(512, return_sequences=True, 
                                  kernel_regularizer=regularizers.l2(0.001)),
        layers.LSTM(512, kernel_regularizer=regularizers.l2(0.001)),
        layers.BatchNormalization(),
        layers.Dropout(0.5),

        # Dense layers for final classification
        layers.Dense(64, activation='relu'),  # First fully connected layer
        layers.Dense(32, activation='relu'),  # Second fully connected layer
        layers.Dense(num_classes, activation='softmax'),  # Output layer
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the k-fold cross-validator
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [21]:
# Perform K-fold Cross-Validation
accuracies = []

for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

    model = create_model(X_train_fold.shape[1:], y_train_fold.shape[1])
    model.fit(X_train_fold, y_train_fold, epochs=10, batch_size=32, verbose=1)
    _, accuracy = model.evaluate(X_val_fold, y_val_fold, verbose=0)
    accuracies.append(accuracy)

average_accuracy = np.mean(accuracies)
print(f'Average Accuracy over {kf.n_splits} folds: {average_accuracy:.4f}')

Epoch 1/10
41/41 [==============================] - 11s 161ms/step - loss: 1.9342 - accuracy: 0.6556
Epoch 2/10
41/41 [==============================] - 6s 142ms/step - loss: 1.3379 - accuracy: 0.7436
Epoch 3/10
41/41 [==============================] - 6s 141ms/step - loss: 1.1595 - accuracy: 0.7591
Epoch 4/10
41/41 [==============================] - 6s 142ms/step - loss: 1.0126 - accuracy: 0.7830
Epoch 5/10
41/41 [==============================] - 6s 142ms/step - loss: 0.9399 - accuracy: 0.7892
Epoch 6/10
41/41 [==============================] - 6s 138ms/step - loss: 0.8982 - accuracy: 0.7761
Epoch 7/10
41/41 [==============================] - 6s 145ms/step - loss: 0.8233 - accuracy: 0.8015
Epoch 8/10
41/41 [==============================] - 6s 141ms/step - loss: 0.7829 - accuracy: 0.8054
Epoch 9/10
41/41 [==============================] - 6s 141ms/step - loss: 0.7993 - accuracy: 0.7792
Epoch 10/10
41/41 [==============================] - 6s 143ms/step - loss: 0.7084 - accuracy: 0.814

In [22]:
# Evaluate the model on the separate test set
_, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f'Test Accuracy: {test_accuracy:.4f}')


13/13 [==============================] - 0s 32ms/step - loss: 0.9178 - accuracy: 0.7704
Test Accuracy: 0.7704


In [47]:
X_train.shape

(1619, 700, 3)

In [48]:
X_train[2]

array([[ -9.123882 , -32.47256  ,  21.128075 ],
       [-10.213771 , -30.90285  ,  19.782068 ],
       [ -9.578003 , -31.999756 ,  20.20885  ],
       ...,
       [ -4.5100293, -39.053997 ,  16.564785 ],
       [ -4.1467347, -38.297512 ,  16.138004 ],
       [ -4.5463605, -38.88379  ,  16.515543 ]], dtype=float32)

In [49]:
# # Define input shape
# input_shape = (700, 3)  # Each input sequence contains 700 time steps with 3 features.

# # Number of categories
# num_classes = 12


In [50]:


# # Define LSTM model
# model = models.Sequential([
#     layers.LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])),
#     layers.Dense(len(np.unique(y)), activation='relu')
# ])

# model.compile(optimizer='adam',
#               loss='sparse_categorical_crossentropy',
#               metrics=['accuracy'])

# model = models.Sequential([
#     layers.Conv1D(filters=8, kernel_size=3, activation='relu', input_shape=input_shape),
#     layers.MaxPooling1D(pool_size=2),
    
#     layers.Conv1D(filters=16, kernel_size=3, activation='relu'),
#     layers.MaxPooling1D(pool_size=2),
    
#     # layers.Flatten(),
#     layers.GRU(32), # flat layer or GRU layer??
    
#     # A small Dense layer acts as the decision layer
#     layers.Dense(16, activation='relu'),
    
#     # Output layer for 12 categories. Using softmax for multi-class classification
#     layers.Dense(num_classes, activation='softmax')
# ])

# GRU model
# model = models.Sequential([
#     # The initial GRU layer. Adjust the number of units and input shape accordingly.
#     layers.GRU(64, activation='tanh', return_sequences=True, input_shape=input_shape),
    
#     # Can add more GRU layers if needed. 
#     # If use multiple GRU layers and want to stack them, ensure the previous layer returns sequences with `return_sequences=True`.
#     layers.GRU(32, activation='tanh', return_sequences=False),
    
#     # A small Dense layer can act as the decision layer
#     layers.Dense(16, activation='relu'),
    
#     # Output layer. Using softmax for multi-class classification
#     layers.Dense(num_classes, activation='softmax')
# ])

# model = models.Sequential([
#     layers.Flatten(input_shape=input_shape),  # Flatten the input data
#     layers.Dense(64, activation='relu'),  # First fully connected layer
#     layers.Dense(32, activation='relu'),  # Second fully connected layer
#     layers.Dense(num_classes, activation='softmax')  # Output layer for 12 categories
# ])

# Compile the model
# model.compile(optimizer='adam', 
#               loss='sparse_categorical_crossentropy', 
#               metrics=['accuracy'])


# model = models.Sequential([
#     layers.GRU(128, input_shape=input_shape, return_sequences=True),  # GRU layer with 128 units
#     layers.GRU(64, return_sequences=False),  # Another GRU layer with 64 units
#     layers.Dense(64, activation='relu'),  # First fully connected layer
#     layers.Dense(32, activation='relu'),  # Second fully connected layer
#     layers.Dense(num_classes, activation='softmax')  # Output layer for 12 categories
# ])

# Compiling the model
# model.compile(
#     optimizer='adam',
#     loss='sparse_categorical_crossentropy',
#     metrics=['accuracy']
# )


# # Train model
# model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2) 


In [51]:
# converter = tf.lite.TFLiteConverter.from_keras_model(model)

# tflite_model_test = converter.convert()

# with open("model.tflite", "wb") as f:
#     f.write(tflite_model_test)

converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Allow for TensorFlow ops that aren't natively supported in TFLite
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS
]

# # Disable the lowering of tensor list operations
converter._experimental_lower_tensor_list_ops = False

tflite_model_test = converter.convert()

with open(exported_path + "model_combined.tflite", "wb") as f:
    f.write(tflite_model_test)


INFO:tensorflow:Assets written to: /var/folders/gg/10wpd3jj5v7dthydfjzl0gcm0000gn/T/tmpf301rzod/assets


INFO:tensorflow:Assets written to: /var/folders/gg/10wpd3jj5v7dthydfjzl0gcm0000gn/T/tmpf301rzod/assets
2023-11-01 12:43:18.302848: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-11-01 12:43:18.302861: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-11-01 12:43:18.302973: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/gg/10wpd3jj5v7dthydfjzl0gcm0000gn/T/tmpf301rzod
2023-11-01 12:43:18.309181: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2023-11-01 12:43:18.309188: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /var/folders/gg/10wpd3jj5v7dthydfjzl0gcm0000gn/T/tmpf301rzod
2023-11-01 12:43:18.332286: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2023-11-01 12:43:18.406428: I tensorflow/cc/saved_model/loader.cc:215] Running initialization